<a href="https://colab.research.google.com/github/stevehooker/welsh-dragons/blob/main/notebooks/comfyui_colab_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ComfyUI on Google Colab (FAST Edition)

**Version 3.2 - 31 Dec 2025**

**⚡ Target startup time: ~10-15 minutes** (down from 30-40 minutes)

## Key Optimisations

1. **Smart Copy** - Copies small custom nodes locally, symlinks large ones (>100MB)
2. **Automatic Dependencies** - Uses ComfyUI-Manager's `restore-dependencies`

**The Problem:** Your `custom_nodes` folder is 6.6 GB with 18,841 files.
- `comfyui_controlnet_aux` alone is 3 GB (model weights, not code)
- Copying all of it would take longer than we'd save

**The Solution:** Copy small nodes (fast imports), symlink large ones (mostly model files anyway).

**Setup Instructions:**
1. Enable GPU: Runtime → Change runtime type → T4 GPU
2. Add secrets (optional): 🔑 icon → Add API keys
3. Run all cells: Runtime → Run all
4. Wait for the cloudflared URL to appear

In [ ]:
#@title 0. Load Secrets (Optional)
import os

try:
    from google.colab import userdata
    secrets_found = []
    for key in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'HF_TOKEN']:
        try:
            value = userdata.get(key)
            if value:
                os.environ[key] = value
                secrets_found.append(key)
        except:
            pass
    if secrets_found:
        print(f"✓ Loaded secrets: {', '.join(secrets_found)}")
    else:
        print("ℹ️  No secrets found. Add them in the 🔑 Secrets panel if needed.")
except Exception as e:
    print(f"ℹ️  Secrets not available: {e}")

In [ ]:
#@title 1. Mount Drive & Setup Paths
import os
import time

print("Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')

# Paths
DRIVE_COMFYUI = "/content/drive/MyDrive/ComfyUI"
LOCAL_COMFYUI = "/content/ComfyUI"

if not os.path.exists(DRIVE_COMFYUI):
    raise RuntimeError(f"❌ ComfyUI not found at {DRIVE_COMFYUI}")

print(f"✓ Drive workspace: {DRIVE_COMFYUI}")
print(f"✓ Local workspace: {LOCAL_COMFYUI}")

In [ ]:
#@title 2. Smart Copy to Local Storage
#@markdown Copies ComfyUI core + small custom nodes locally.
#@markdown Large nodes (>100MB) are symlinked to avoid long copy times.

import shutil
import os
import time

#@markdown ---
#@markdown **Size threshold for symlinking (MB):**
SIZE_THRESHOLD_MB = 100  #@param {type:"integer"}

start_time = time.time()

def get_folder_size_mb(path):
    """Get folder size in MB (quick estimate using du)"""
    try:
        import subprocess
        result = subprocess.run(['du', '-sm', path], capture_output=True, text=True, timeout=30)
        if result.returncode == 0:
            return int(result.stdout.split()[0])
    except:
        pass
    return 0

# Clean any previous setup
if os.path.exists(LOCAL_COMFYUI):
    print("Cleaning previous local copy...")
    shutil.rmtree(LOCAL_COMFYUI)

os.makedirs(LOCAL_COMFYUI, exist_ok=True)

print("=" * 70)
print("Smart Copy: ComfyUI to Local Storage")
print(f"Nodes > {SIZE_THRESHOLD_MB}MB will be symlinked, smaller ones copied")
print("=" * 70)

# Copy ComfyUI core (not custom_nodes yet)
core_items = [
    'main.py', 'requirements.txt', 'comfy', 'comfy_extras',
    'web', 'script_examples', 'user', 'notebooks', 'app', '.git',
    'execution.py', 'server.py', 'folder_paths.py', 'nodes.py',
    'node_helpers.py', 'latent_preview.py', 'extra_model_paths.yaml',
]

print("\n[1/3] Copying ComfyUI core...")
for item in core_items:
    src = os.path.join(DRIVE_COMFYUI, item)
    dst = os.path.join(LOCAL_COMFYUI, item)
    if os.path.exists(src):
        if os.path.isdir(src):
            shutil.copytree(src, dst, symlinks=True)
        else:
            shutil.copy2(src, dst)
print(f"  ✓ Core copied")

# Smart copy custom_nodes
print("\n[2/3] Processing custom_nodes (smart copy)...")
src_custom = os.path.join(DRIVE_COMFYUI, 'custom_nodes')
dst_custom = os.path.join(LOCAL_COMFYUI, 'custom_nodes')
os.makedirs(dst_custom, exist_ok=True)

copied_nodes = []
symlinked_nodes = []
total_copied_mb = 0

if os.path.exists(src_custom):
    nodes = sorted(os.listdir(src_custom))
    for node in nodes:
        src_node = os.path.join(src_custom, node)
        dst_node = os.path.join(dst_custom, node)
        
        if not os.path.isdir(src_node):
            # Copy single files
            shutil.copy2(src_node, dst_node)
            continue
        
        # Check size
        size_mb = get_folder_size_mb(src_node)
        
        if size_mb > SIZE_THRESHOLD_MB:
            # Symlink large nodes
            os.symlink(src_node, dst_node)
            symlinked_nodes.append((node, size_mb))
            print(f"  🔗 {node} ({size_mb} MB) → symlinked")
        else:
            # Copy small nodes
            shutil.copytree(src_node, dst_node, symlinks=True)
            copied_nodes.append((node, size_mb))
            total_copied_mb += size_mb

print(f"\n  Summary:")
print(f"  • Copied: {len(copied_nodes)} nodes ({total_copied_mb} MB)")
print(f"  • Symlinked: {len(symlinked_nodes)} large nodes")

# Symlink model directories
print("\n[3/3] Symlinking model directories...")
for dir_name in ['models', 'input', 'output']:
    src = os.path.join(DRIVE_COMFYUI, dir_name)
    dst = os.path.join(LOCAL_COMFYUI, dir_name)
    if os.path.exists(src) and not os.path.exists(dst):
        os.symlink(src, dst)
        print(f"  ✓ {dir_name} → Drive")

elapsed = time.time() - start_time
print(f"\n{'='*70}")
print(f"✓ Setup complete in {elapsed:.1f} seconds")
print(f"\nSmall nodes load from LOCAL SSD (fast!)")
print(f"Large nodes load from Drive (slower, but no copy wait)")

In [ ]:
#@title 3. Install Dependencies (Auto-detected)
#@markdown Uses ComfyUI-Manager to scan all custom nodes and install their requirements.

import time
start = time.time()

print("=" * 60)
print("Installing Dependencies")
print("=" * 60)

%cd {LOCAL_COMFYUI}

# 1. Core ComfyUI requirements
print("\n[1/3] Core ComfyUI requirements...")
!pip install -q -r requirements.txt
!pip install -q --upgrade comfyui-frontend-package

# 2. Auto-detect and install all custom node dependencies
print("\n[2/3] Scanning custom nodes for dependencies...")
print("       (cm-cli restore-dependencies)")
!python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies

# 3. Final cleanup
print("\n[3/3] Final cleanup...")
!pip install -q --upgrade comfyui-frontend-package

print(f"\n✓ All dependencies installed in {time.time()-start:.1f}s")

In [ ]:
#@title 4. Verify GPU
import torch

print(f"PyTorch {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print("\n✓ GPU ready!")
else:
    print("\n❌ No GPU! Go to: Runtime > Change runtime type > T4 GPU")

In [ ]:
#@title 5. Start ComfyUI
import subprocess
import threading
import time
import socket
import re
import os

# Download cloudflared
![ ! -f ~/cloudflared-linux-amd64.deb ] && wget -q -O ~/cloudflared-linux-amd64.deb https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb 2>/dev/null

cloudflared_url = None
comfyui_ready = False
start_time = time.time()

def start_tunnel(port=8188):
    """Start cloudflared and capture URL silently"""
    global cloudflared_url
    
    for _ in range(120):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex(('127.0.0.1', port))
            sock.close()
            if result == 0:
                break
        except:
            pass
        time.sleep(0.5)
    
    proc = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        bufsize=1
    )
    
    for line in proc.stderr:
        if "trycloudflare.com" in line:
            match = re.search(r'https://[a-z0-9-]+\.trycloudflare\.com', line)
            if match:
                cloudflared_url = match.group(0)
                break

tunnel_thread = threading.Thread(target=start_tunnel, daemon=True)
tunnel_thread.start()

%cd {LOCAL_COMFYUI}

print("=" * 70)
print("Starting ComfyUI (hybrid: local code + Drive models)")
print("=" * 70)
print()

proc = subprocess.Popen(
    ["python", "main.py", "--dont-print-server"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

for line in proc.stdout:
    print(line, end='')
    
    if "Import times for custom nodes:" in line and not comfyui_ready:
        comfyui_ready = True
        time.sleep(2)
        
        elapsed = time.time() - start_time
        
        if cloudflared_url:
            print("\n" + "=" * 70)
            print(f"\n🎉 COMFYUI READY in {elapsed:.0f} seconds!")
            print(f"\n🌐 ACCESS HERE: {cloudflared_url}")
            print("\n" + "=" * 70 + "\n")
        else:
            print("\n⚠️  Waiting for tunnel URL...")
            for _ in range(15):
                if cloudflared_url:
                    print("\n" + "=" * 70)
                    print(f"\n🎉 COMFYUI READY in {elapsed:.0f} seconds!")
                    print(f"\n🌐 ACCESS HERE: {cloudflared_url}")
                    print("\n" + "=" * 70 + "\n")
                    break
                time.sleep(1)

## How Smart Copy Works

### The Problem
Your `custom_nodes` folder is **6.6 GB** with **18,841 files**:
- `comfyui_controlnet_aux`: 2.97 GB (45%) — mostly model weights
- `Bjornulf_custom_nodes`: 574 MB
- `RES4LYF`: 473 MB
- Plus ~60 other nodes

Copying all 6.6 GB would take ~5-10 minutes — longer than we'd save.

### The Solution
**Smart copy by size threshold (default: 100 MB)**

| Node Size | Action | Why |
|-----------|--------|-----|
| < 100 MB | Copy to local | Fast imports, quick copy |
| > 100 MB | Symlink to Drive | Avoids long copy, mostly model files anyway |

### Expected Results
- **~50 small nodes** copied locally (~500 MB, ~1-2 min)
- **~10 large nodes** symlinked (instant)
- Small nodes load 10x faster
- Large nodes load at Drive speed (but they were slow anyway due to size)

### Tuning
Adjust `SIZE_THRESHOLD_MB` in Section 2:
- **Lower (50 MB)**: More nodes copied, longer setup, faster runtime
- **Higher (200 MB)**: Faster setup, more nodes on Drive